### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7fe381346050>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7fe3813460e0>
layer: Linear
func: <function parse_linear_layer at 0x7fe3813463b0>
layer: Softmax
func: <function parse_activation_layer at 0x7fe381346440>
layer: ReLU
func: <function parse_activation_layer at 0x7fe381346440>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7fe3813464d0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7fe3813464d0>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7fe3813427a0>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7fe381342950>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7fe381342a70>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7fe381342830>
layer: NodeEncoder
func: <f

/home/swissman777/anaconda3/envs/hls4ml/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40).eval()
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)
new_state_dict = OrderedDict()
for key in torch_model.O.layers[1].state_dict():
#     print(torch_model.O.layers[1].state_dict()[key].shape)
    if key != "num_batches_tracked":
        new_state_dict[key] = torch.rand(40)

torch_model.O.layers[1].load_state_dict(new_state_dict)
print(torch_model.O.layers[1].state_dict())
# for name, submodule in torch_model.named_modules():
#     if name != "":
#         print(f"{name}: {submodule}")

OrderedDict([('weight', tensor([0.25492030, 0.24551541, 0.09850901, 0.50610167, 0.40471417, 0.74222189,
        0.78643876, 0.45589358, 0.31176186, 0.46250528, 0.60469770, 0.53806579,
        0.16209030, 0.42769414, 0.25744629, 0.43551594, 0.17785794, 0.84629416,
        0.78365254, 0.13654888, 0.92400128, 0.35559273, 0.06362355, 0.06826925,
        0.78546554, 0.02573627, 0.18891090, 0.14141375, 0.25868684, 0.82461905,
        0.92814422, 0.66418916, 0.37154651, 0.68763214, 0.55344772, 0.55224824,
        0.87121969, 0.14227891, 0.22192371, 0.61809975])), ('bias', tensor([0.15977389, 0.72546005, 0.97197855, 0.82563680, 0.07433015, 0.91807199,
        0.42197108, 0.18261266, 0.79096532, 0.11385942, 0.89414120, 0.26867706,
        0.49521214, 0.39871615, 0.20885032, 0.68721026, 0.46592551, 0.86679852,
        0.72296393, 0.08661199, 0.81924605, 0.88838691, 0.81033218, 0.29575837,
        0.58426809, 0.04706109, 0.29182684, 0.41725045, 0.39764506, 0.25243640,
        0.72814095, 0.368973

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_attr": 3,
        "node_dim": common_dim,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<52,20>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid', #sigmoid
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
pyg_to_hls config: {'OutputDir': 'test_GNN', 'ProjectName': 'myproject', 'Backend': 'Vivado', 'XilinxPart': 'xcku115-flvb2104-2-i', 'Board': None, 'ClockPeriod': 5, 'IOType': 'io_parallel', 'HLSConfig': {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}, 'PytorchModel': InteractionNetwork(), 'InputShape': {'NodeAttr': [28, 3], 'EdgeAttr': [37, 4], 'EdgeIndex': [37, 2], 'NodeDim': 5, 'EdgeDim': 5}, 'ForwardDictionary': OrderedDict([('node_encoder', 'NodeEncoder'), ('edge_encoder', 'EdgeEncoder'), ('O', 'NodeBlock')]), 'ActivateFinal': 'sigmoid', 'ParallelizationFactor': 16, 'gnn_resource_limit': 'false'}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
node_encoder la

## hls_model.compile() builds the C-function for the model

In [7]:
hls_model.compile()

Writing HLS project
Layer self.weights: OrderedDict()
Layer self.weights: OrderedDict()
Layer self.weights: OrderedDict()
Layer self.weights: OrderedDict([('weight', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe3683c6350>), ('bias', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe3683bde50>)])
Layer self.weights: OrderedDict([('weight', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368436d10>), ('bias', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368436bd0>)])
Layer self.weights: OrderedDict()
Layer self.weights: OrderedDict([('O_w0', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368436890>), ('O_norm_s0', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368421710>), ('O_norm_b0', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe3683bd490>), ('O_w1', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368412cd0>), ('O_w2', <hls4ml.model.hls_layers.WeightVariable object at 0x7fe368412d10>), ('O_w3', <hls4ml.model.hls_layers.We

Done
lib_name: firmware/myproject-2B8c45ae.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [8]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [9]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

# print((np.abs(torch_pred- hls_pred)))
# print(torch_pred)
# print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
torch pred shape: (140,)
hls_pred.shape: (140,)
MSE: 4.010399607068393e-06


In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [11]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

MSE: 4.010399607068393e-06
MSE: 4.760589945362881e-06
MSE: 4.918084869132144e-06
MSE: 4.087110028194729e-06
MSE: 4.722618996311212e-06
MSE: 4.883581823378336e-06
MSE: 4.465672645892482e-06
MSE: 4.2049173316627275e-06
MSE: 4.3018303585995454e-06
MSE: 4.487245860218536e-06
MSE_l: [4.0103996e-06 4.7605899e-06 4.9180849e-06 4.0871100e-06 4.7226190e-06
 4.8835818e-06 4.4656726e-06 4.2049173e-06 4.3018304e-06 4.4872459e-06]
overall MSE: 4.484204964683158e-06
(10,)


In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =10000): #n_graphs =1000
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array

        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        else:
            print("bad graph")
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7fe3683fba90>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7fe3683f3890>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7fe3683f3510>}


In [15]:
# from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
# node_scaler = StandardScaler()
# edge_scaler = StandardScaler()
# for stage in [stages[0]]:
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

#     for data in graphs: # fit data to scaler b4 feedforwarding
# #         print(f"data type: {type(data)}")
#         node_scaler.fit(data.x) 
#         edge_scaler.fit(data.edge_attr) 
    
    for data in graphs:
       
#         data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
#         data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data[0] = node_scaler.transform(data.hls_data[0])
#         data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
#         print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
#         print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
#         break
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 10000
writing test bench data for 1st graph
MSE: 2.3031766431813594e-06
MSE: 2.656084461705177e-06
MSE: 5.524444190996292e-07
MSE: 2.0140012111369288e-06
MSE: 2.890889845730271e-06
MSE: 1.429610620107269e-06
MSE: 2.9358006941038184e-06
MSE: 3.1524448331765598e-06
MSE: 2.0140266769885784e-06
MSE: 2.1601617845590226e-06
MSE: 3.816875960183097e-06
MSE: 2.014077153944527e-06
MSE: 2.749530722212512e-06
MSE: 3.3897747471201e-06
MSE: 3.0558385333279148e-06
MSE: 2.1603038931061747e-06
MSE: 2.994289161506458e-06
MSE: 2.5979813926824136e-06
MSE: 2.7411383598519024e-06
MSE: 3.0369496926141437e-06
MSE: 3.523455234244466e-06
MSE: 2.339808133910992e-06
MSE: 3.0917492495063925e-06
MSE: 3.244689196435502e-06
MSE: 2.8892654881929047e-06
MSE: 3.228837840651977e-06
MSE: 2.014077153944527e-06
MSE: 2.4506450699846027e-06
MSE: 9.910403377944021e-07
MSE: 2.3039312964101555e-06
MSE: 1.283412075281376e-06
MSE: 2.7389239676267607e-06
MSE: 1.4294593029262614e-06
MSE: 1.7217299728145008e-06
MSE: 2.17006

MSE: 2.6080374482262414e-06
MSE: 2.3870072709542e-06
MSE: 3.3819303553173086e-06
MSE: 2.9385209927568212e-06
MSE: 2.450482497806661e-06
MSE: 2.59823605119891e-06
MSE: 3.2384218684455846e-06
MSE: 2.3063480512064416e-06
MSE: 2.5985380034398986e-06
MSE: 3.186537696819869e-06
MSE: 9.910655762723763e-07
MSE: 3.376918130015838e-06
MSE: 1.721729859127663e-06
MSE: 2.4522680632799165e-06
MSE: 3.086676997554605e-06
MSE: 4.0657414501765743e-07
MSE: 1.4295097798822098e-06
MSE: 2.4507598936907016e-06
MSE: 2.4524456421204377e-06
MSE: 1.5754989135530195e-06
MSE: 2.621640760480659e-06
MSE: 2.0138750187470578e-06
MSE: 1.4295854953161324e-06
MSE: 2.5955641831387766e-06
MSE: 3.158565505145816e-06
MSE: 1.1425273527265745e-07
MSE: 2.744721314229537e-06
MSE: 3.525098009049543e-06
MSE: 2.327009042346617e-06
MSE: 3.2300574730470544e-06
MSE: 2.596219928818755e-06
MSE: 2.740961235758732e-06
MSE: 2.0191719158901833e-06
MSE: 2.452521130180685e-06
MSE: 2.013900257225032e-06
MSE: 2.0139257230766816e-06
MSE: 2.67473

MSE: 3.5235998439020477e-06
MSE: 2.452496119076386e-06
MSE: 2.890889845730271e-06
MSE: 2.3063480512064416e-06
MSE: 2.8951842523383675e-06
MSE: 2.014000756389578e-06
MSE: 2.5965941858885344e-06
MSE: 9.909643949868041e-07
MSE: 3.0840681120025693e-06
MSE: 2.1601997559628217e-06
MSE: 2.0732741177198477e-06
MSE: 2.199878281317069e-06
MSE: 3.815845502685988e-06
MSE: 1.4294591892394237e-06
MSE: 2.014102392422501e-06
MSE: 2.9374423320405185e-06
MSE: 2.013925495703006e-06
MSE: 2.6871791760640917e-06
MSE: 2.4524456421204377e-06
MSE: 2.3062218588165706e-06
MSE: 2.678302962522139e-06
MSE: 2.0910731564072194e-06
MSE: 2.160003987228265e-06
MSE: 2.1575658593064873e-06
MSE: 2.939970045190421e-06
MSE: 2.1668249701178866e-06
MSE: 3.060220024053706e-06
MSE: 2.45254682340601e-06
MSE: 1.5757083247081027e-06
MSE: 2.0138750187470578e-06
MSE: 2.452268745400943e-06
MSE: 2.5978310986829456e-06
MSE: 2.6124512260139454e-06
MSE: 2.595209934952436e-06
MSE: 2.3420218440151075e-06
MSE: 2.595846126496326e-06
MSE: 2.01

MSE: 1.7561991398906684e-06
MSE: 2.1600735635729507e-06
MSE: 2.459252755215857e-06
MSE: 1.28326018966618e-06
MSE: 1.575567353029328e-06
MSE: 2.0139257230766816e-06
MSE: 1.1371125765435863e-06
MSE: 2.530765641495236e-06
MSE: 1.429610620107269e-06
MSE: 3.229727326470311e-06
MSE: 2.304847839695867e-06
MSE: 3.0834485187369864e-06
MSE: 2.754424031081726e-06
MSE: 2.936858209068305e-06
MSE: 1.8679033928492572e-06
MSE: 1.140506853403167e-07
MSE: 2.890786163334269e-06
MSE: 1.1371125765435863e-06
MSE: 2.1609064333460992e-06
MSE: 2.452268745400943e-06
MSE: 3.0811704618827207e-06
MSE: 2.5979670681408606e-06
MSE: 3.1831357318878872e-06
MSE: 1.4295854953161324e-06
MSE: 2.7439455152489245e-06
MSE: 3.6683279631688492e-06
MSE: 1.1420222278957226e-07
MSE: 3.5234561437391676e-06
MSE: 2.9057171104796e-06
MSE: 3.094818794124876e-06
MSE: 2.7448174932942493e-06
MSE: 1.5757334494992392e-06
MSE: 3.3750609418348176e-06
MSE: 2.160098802050925e-06
gr: 3
CONFIG_T::aggr: 3
CONFIG_T::aggr: 3
CONFIG_T::aggr: 3
CONFIG

MSE: 2.3063480512064416e-06
MSE: 2.1603480035992106e-06
MSE: 2.454401283102925e-06
MSE: 8.448920993942011e-07
MSE: 5.526464406102605e-07
MSE: 2.1600230866170023e-06
MSE: 3.123131136817392e-06
MSE: 2.1582388853857992e-06
MSE: 1.5755819049445563e-06
MSE: 2.598568471512408e-06
MSE: 3.669722673294018e-06
MSE: 3.083426236116793e-06
MSE: 1.5754051219118992e-06
MSE: 2.5988676952692913e-06
MSE: 2.9390155305009102e-06
MSE: 2.7921405489905737e-06
MSE: 2.5992951577791246e-06
MSE: 3.819666744675487e-06
MSE: 2.1600230866170023e-06
MSE: 2.5984111289290013e-06
MSE: 2.9127827474439982e-06
MSE: 2.451743512210669e-06
MSE: 3.3800040455389535e-06
MSE: 1.7215593288710807e-06
MSE: 2.3195873382064747e-06
MSE: 2.1577932329819305e-06
MSE: 2.906620011344785e-06
MSE: 2.7902217425435083e-06
MSE: 3.229182084396598e-06
MSE: 3.4573979519336717e-06
MSE: 2.306272335772519e-06
MSE: 2.890889845730271e-06
MSE: 3.0533390145137673e-06
MSE: 3.230548827559687e-06
MSE: 2.890839368774323e-06
MSE: 2.622229658300057e-06
MSE: 3.8

...

MSE: 1.8679033928492572e-06
MSE: 3.0948651783546666e-06
MSE: 2.086784434141009e-06
MSE: 2.450136207698961e-06
MSE: 2.0139179923717165e-06
MSE: 1.8677772004593862e-06
MSE: 2.8879387627966935e-06
MSE: 2.5970205115299905e-06
MSE: 2.452268745400943e-06
MSE: 1.2832857692046673e-06
MSE: 2.7908245101571083e-06
MSE: 2.7440246412879787e-06
MSE: 2.160098802050925e-06
MSE: 1.7217047343365266e-06
MSE: 3.036074076590012e-06
MSE: 1.7217030290339608e-06
MSE: 1.8677728803595528e-06
MSE: 1.1373397228453541e-06
MSE: 3.2282944175676676e-06
MSE: 3.2281134281220147e-06
MSE: 2.7448174932942493e-06
MSE: 1.5755300637465552e-06
MSE: 1.575531427988608e-06
MSE: 2.45254682340601e-06
MSE: 2.4224530079663964e-06
MSE: 2.378802946623182e-06
MSE: 1.5754051219118992e-06
MSE: 2.88868841380463e-06
MSE: 1.8679033928492572e-06
MSE: 3.18017464451259e-06
MSE: 1.5756829725432908e-06
MSE: 1.1417696299531599e-07
MSE: 2.943391791632166e-06
MSE: 1.8678023252505227e-06
MSE: 2.598228775241296e-06
MSE: 2.4525895696569933e-06
MSE: 2.

MSE: 1.429358235327527e-06
MSE: 2.155215724997106e-06
MSE: 2.1602252218144713e-06
MSE: 2.172565473301802e-06
MSE: 2.3054440134728793e-06
MSE: 2.5977587938541546e-06
MSE: 1.575531427988608e-06
MSE: 2.9375737540249247e-06
MSE: 2.3064490051183384e-06
MSE: 2.950900352516328e-06
MSE: 2.621075509523507e-06
MSE: 8.449931101495167e-07
MSE: 6.985924301261548e-07
MSE: 1.5755061895106337e-06
MSE: 2.3038307972456096e-06
MSE: 9.911160532283247e-07
MSE: 2.3440543372998945e-06
MSE: 2.5231745439668885e-06
MSE: 2.4490880150551675e-06
MSE: 1.8677772004593862e-06
MSE: 2.200051085310406e-06
MSE: 1.721831154100073e-06
MSE: 2.3052327833283925e-06
MSE: 2.6053930923808366e-06
MSE: 1.5754808373458218e-06
MSE: 2.597236743895337e-06
MSE: 2.306373289684416e-06
MSE: 1.721831154100073e-06
MSE: 9.91141291706299e-07
MSE: 1.5757083247081027e-06
MSE: 3.010891532539972e-06
MSE: 2.3062468699208694e-06
MSE: 2.0140012111369288e-06
MSE: 3.0844089451420587e-06
MSE: 2.72892293651239e-06
MSE: 1.4295349046733463e-06
MSE: 2.0140

MSE: 1.867701371338626e-06
MSE: 2.0139971184107708e-06
MSE: 2.3062468699208694e-06
MSE: 3.1118543120101094e-06
MSE: 2.597088496258948e-06
MSE: 2.650281885507866e-06
MSE: 2.7929386305913795e-06
MSE: 1.4294340644482872e-06
MSE: 2.4594000933575444e-06
MSE: 2.766223815342528e-06
MSE: 2.4511605261068325e-06
MSE: 2.604008102480293e-07
MSE: 1.8678978221942089e-06
MSE: 1.1458105575457012e-07
MSE: 1.8677518482945743e-06
MSE: 1.8678840660868445e-06
MSE: 3.038713430214557e-06
MSE: 2.5226363504771143e-06
MSE: 2.3903578494355315e-06
MSE: 9.910403377944021e-07
MSE: 2.4850778572726995e-06
MSE: 1.5757334494992392e-06
MSE: 2.1585572085314197e-06
MSE: 5.52570668332919e-07
MSE: 2.8909150842082454e-06
MSE: 1.9528765733412e-06
MSE: 3.2315238058799878e-06
MSE: 2.9857008030376164e-06
MSE: 2.160098802050925e-06
MSE: 2.0394336388562806e-06
MSE: 1.721728267511935e-06
MSE: 2.6244845230394276e-06
MSE: 2.1602252218144713e-06
MSE: 2.3596035134687554e-06
MSE: 2.0140473679930437e-06
MSE: 3.128486696368782e-06
MSE: 1.

MSE: 3.3811807043093722e-06
MSE: 2.1601997559628217e-06
MSE: 3.3935530154849403e-06
MSE: 3.2297364214173285e-06
MSE: 2.3063275875756517e-06
MSE: 2.452442686262657e-06
MSE: 2.9859716050850693e-06
MSE: 1.575540409248788e-06
MSE: 2.3056729787640506e-06
MSE: 2.1620228380925255e-06
MSE: 1.5757334494992392e-06
MSE: 1.1373145980542176e-06
MSE: 2.3048253297019983e-06
MSE: 2.4525213575543603e-06
MSE: 2.7473338377603795e-06
MSE: 2.5984420517488616e-06
MSE: 1.1372387689334573e-06
MSE: 1.8677513935472234e-06
MSE: 3.0865710414218483e-06
MSE: 2.5967055989895016e-06
MSE: 2.598568471512408e-06
MSE: 2.598568471512408e-06
MSE: 1.2832100537707447e-06
MSE: 3.376774429852958e-06
MSE: 1.1372892458894057e-06
MSE: 2.1600230866170023e-06
MSE: 2.0140266769885784e-06
MSE: 1.7217047343365266e-06
MSE: 3.0831172352918657e-06
MSE: 6.986681455600774e-07
MSE: 2.395232513663359e-06
MSE: 1.283488018088974e-06
MSE: 2.014000756389578e-06
MSE: 3.3774549592635594e-06
MSE: 1.1371630534995347e-06
MSE: 4.0657414501765743e-07
M

MSE: 1.8678276774153346e-06
MSE: 2.3061459160089726e-06
MSE: 2.3293648609978845e-06
MSE: 2.3056484224071028e-06
MSE: 2.014102392422501e-06
MSE: 1.8677772004593862e-06
MSE: 3.259790219090064e-06
MSE: 2.598448872959125e-06
MSE: 1.5755792901472887e-06
MSE: 1.867852006398607e-06
MSE: 2.4525213575543603e-06
MSE: 3.815689069597283e-06
MSE: 2.014051688092877e-06
MSE: 2.182955086027505e-06
MSE: 2.937560566351749e-06
MSE: 2.3031063847156474e-06
MSE: 3.2305556487699505e-06
MSE: 2.5984752483054763e-06
MSE: 2.743967797869118e-06
MSE: 2.306243004568387e-06
MSE: 3.085223852394847e-06
MSE: 9.910655762723763e-07
MSE: 2.16025023291877e-06
MSE: 2.159469431717298e-06
MSE: 2.8489509986684425e-06
MSE: 1.7217553249793127e-06
MSE: 1.5757083247081027e-06
MSE: 8.446897936664755e-07
MSE: 1.8678023252505227e-06
MSE: 2.0140175820415607e-06
MSE: 2.938449142675381e-06
MSE: 2.9597506454592803e-06
MSE: 8.449173378721753e-07
MSE: 2.0139759726589546e-06
MSE: 2.5980809823522577e-06
MSE: 2.0139759726589546e-06
MSE: 3.082

In [16]:
MSE_l.shape

(30000,)

In [17]:
# # torch_model.state_dict()
# with open('MSE_l.npy', 'wb') as f:
#     np.save(f, MSE_l)